In [1]:
import os
import xarray as xr
import pandas as pd
import numpy as np
import geopandas as gpd
import glob 
import pytz

In [2]:
# Set the folder path where your .nc files are located
folder_path = '/home/paulc600/local/Easymore/PRMS_merged_easymore/*.nc'

In [42]:
# Specify the file paths and names
precipitation_file_path = '/home/paulc600/scratch/PRMS/02_directory/inputs/precip_m.day'
temperature_max_file_path = '/home/paulc600/scratch/PRMS/02_directory/inputs/tmax.day'
temperature_min_file_path = '/home/paulc600/scratch/PRMS/02_directory/inputs/tmin.day'

In [4]:
# Get a list of all .nc files in the folder
file_list = glob.glob(folder_path)

In [5]:
# Open the .nc files using xarray's open_mfdataset
dataset = xr.open_mfdataset(file_list, combine='by_coords')

In [6]:
# Extract the time coordinate from the dataset
time_coords = dataset.coords['time']

In [7]:
# Convert the object array to pandas Timestamp objects
time_values = pd.to_datetime(time_coords.data)

In [8]:
# Define the UTC and MST time zones
utc_tz = pytz.UTC
mst_tz = pytz.timezone('US/Mountain')

In [9]:
# Convert time from UTC to MST using pandas Timestamp
converted_times = [utc_tz.localize(dt).astimezone(mst_tz) for dt in time_values]

In [10]:
# Create a new dataset with the converted time coordinate
ds_converted = dataset.assign_coords(time=converted_times)

In [11]:
ds_converted

<xarray.Dataset>
Dimensions:        (time: 341496, ID: 909)
Coordinates:
  * time           (time) object 1980-01-01T06:00:00-07:00 ... 2018-12-31T05:...
  * ID             (ID) float64 9.596e+04 9.597e+04 ... 9.701e+04 9.701e+04
Data variables:
    latitude       (time, ID) float64 dask.array<chunksize=(8760, 909), meta=np.ndarray>
    longitude      (time, ID) float64 dask.array<chunksize=(8760, 909), meta=np.ndarray>
    temperature    (time, ID) float32 dask.array<chunksize=(8760, 909), meta=np.ndarray>
    precipitation  (time, ID) float32 dask.array<chunksize=(8760, 909), meta=np.ndarray>
Attributes:
    CDI:          Climate Data Interface version 2.0.5 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    Author:       The data were written by author name
    History:      Created Thu Jun 22 11:17:53 2023
    history:      Tue Jun 27 12:26:26 2023: cdo mergetime PRMS_easymore_remap...
    CDO:          Climate Data Operators version 2.0.5 (https://mpimet.mpg.de...

In [12]:
precipitation_daily = ds_converted['precipitation'].resample(time='D').sum(dim='time')

In [13]:
precipitation_daily

<xarray.DataArray 'precipitation' (time: 14245, ID: 909)>
dask.array<stack, shape=(14245, 909), dtype=float32, chunksize=(1, 909), chunktype=numpy.ndarray>
Coordinates:
  * ID       (ID) float64 9.596e+04 9.597e+04 9.597e+04 ... 9.701e+04 9.701e+04
  * time     (time) object 315558000000000000 ... 1546239600000000000
Attributes:
    long_name:  Analysis: Quantity of precipitation
    units:      m

In [14]:
# Resample the temperature array to daily maximum and minimum
temperature_daily_max = ds_converted['temperature'].resample(time='D').max(dim='time')
temperature_daily_min = ds_converted['temperature'].resample(time='D').min(dim='time')

In [15]:
# Convert the resampled arrays to pandas DataFrames
precipitation_daily_df = precipitation_daily.to_dataframe().unstack('ID')['precipitation']
temperature_daily_max_df = temperature_daily_max.to_dataframe().unstack('ID')['temperature']
temperature_daily_min_df = temperature_daily_min.to_dataframe().unstack('ID')['temperature']

In [16]:
precipitation_daily_df

ID,95965.0,95966.0,95968.0,95969.0,95970.0,95974.0,95977.0,95979.0,95980.0,95981.0,...,97351.0,97352.0,97353.0,96100.0,96102.0,96139.0,96959.0,96978.0,97007.0,97013.0
time,,,,,,,,,,,,,,,,,,,,,
1980-01-01 00:00:00-07:00,0.000743,0.000770,0.000938,0.000938,0.000887,0.000846,0.000276,0.000121,0.000629,0.000094,...,3.595683e-06,6.261100e-07,6.291972e-07,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1980-01-02 00:00:00-07:00,0.000595,0.000730,0.000967,0.000967,0.000977,0.000862,0.000233,0.000210,0.001029,0.000152,...,8.582811e-05,4.382972e-05,5.301153e-05,0.000064,0.000069,0.000056,2.451854e-04,1.997415e-04,1.662830e-04,1.203565e-04
1980-01-03 00:00:00-07:00,0.004457,0.004424,0.005175,0.005175,0.004883,0.004565,0.000458,0.000374,0.003415,0.000208,...,1.392809e-04,2.698785e-04,2.652397e-04,0.000004,0.000004,0.000017,1.805580e-07,5.937065e-08,3.028520e-07,4.281175e-07
1980-01-04 00:00:00-07:00,0.000480,0.000449,0.000369,0.000369,0.000339,0.000342,0.000205,0.000160,0.000188,0.000094,...,1.447320e-04,1.715823e-04,8.390634e-05,0.000000,0.000000,0.000000,1.311062e-05,4.979006e-06,0.000000e+00,0.000000e+00
1980-01-05 00:00:00-07:00,0.017046,0.016897,0.017571,0.017571,0.017485,0.017176,0.014209,0.012255,0.017053,0.012251,...,3.647967e-03,5.177778e-03,4.166152e-03,0.012274,0.012070,0.013187,8.091788e-03,8.580540e-03,9.541063e-03,1.044318e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-27 00:00:00-07:00,0.000680,0.000643,0.000655,0.000655,0.000669,0.000626,0.000402,0.000416,0.000740,0.000362,...,1.931212e-04,2.452591e-04,2.125588e-04,0.000520,0.000516,0.000557,4.105257e-04,4.283622e-04,4.876879e-04,5.125729e-04
2018-12-28 00:00:00-07:00,0.000416,0.000421,0.000420,0.000420,0.000404,0.000397,0.000237,0.000144,0.000320,0.000135,...,5.797572e-05,2.086436e-05,3.461967e-05,0.000000,0.000000,0.000000,1.449955e-07,2.026077e-07,8.678600e-08,2.029225e-09
2018-12-29 00:00:00-07:00,0.012657,0.014018,0.017795,0.017795,0.017351,0.016241,0.006376,0.002629,0.015110,0.002597,...,1.372970e-07,1.154708e-06,2.652976e-06,0.000467,0.000451,0.000399,1.812913e-04,2.044448e-04,2.481727e-04,3.005849e-04


In [17]:
# Reset the index to remove time and column names from the index
precipitation_daily_df = precipitation_daily_df.reset_index()
precipitation_daily_df.loc[-1] = precipitation_daily_df.columns
precipitation_daily_df.sort_index(inplace=True)

temperature_daily_max_df = temperature_daily_max_df.reset_index()
temperature_daily_max_df.loc[-1] = temperature_daily_max_df.columns
temperature_daily_max_df.sort_index(inplace=True)

temperature_daily_min_df = temperature_daily_min_df.reset_index()
temperature_daily_min_df.loc[-1] = temperature_daily_min_df.columns
temperature_daily_min_df.sort_index(inplace=True)


In [18]:
precipitation_daily_df

ID,time,95965.0,95966.0,95968.0,95969.0,95970.0,95974.0,95977.0,95979.0,95980.0,...,97351.0,97352.0,97353.0,96100.0,96102.0,96139.0,96959.0,96978.0,97007.0,97013.0
-1,time,95965.000000,95966.000000,95968.000000,95969.000000,95970.000000,95974.000000,95977.000000,95979.000000,95980.000000,...,9.735100e+04,9.735200e+04,9.735300e+04,96100.000000,96102.000000,96139.000000,9.695900e+04,9.697800e+04,9.700700e+04,9.701300e+04
0,1980-01-01 00:00:00-07:00,0.000743,0.000770,0.000938,0.000938,0.000887,0.000846,0.000276,0.000121,0.000629,...,3.595683e-06,6.261100e-07,6.291972e-07,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,1980-01-02 00:00:00-07:00,0.000595,0.000730,0.000967,0.000967,0.000977,0.000862,0.000233,0.000210,0.001029,...,8.582811e-05,4.382972e-05,5.301153e-05,0.000064,0.000069,0.000056,2.451854e-04,1.997415e-04,1.662830e-04,1.203565e-04
2,1980-01-03 00:00:00-07:00,0.004457,0.004424,0.005175,0.005175,0.004883,0.004565,0.000458,0.000374,0.003415,...,1.392809e-04,2.698785e-04,2.652397e-04,0.000004,0.000004,0.000017,1.805580e-07,5.937065e-08,3.028520e-07,4.281175e-07
3,1980-01-04 00:00:00-07:00,0.000480,0.000449,0.000369,0.000369,0.000339,0.000342,0.000205,0.000160,0.000188,...,1.447320e-04,1.715823e-04,8.390634e-05,0.000000,0.000000,0.000000,1.311062e-05,4.979006e-06,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14240,2018-12-27 00:00:00-07:00,0.000680,0.000643,0.000655,0.000655,0.000669,0.000626,0.000402,0.000416,0.000740,...,1.931212e-04,2.452591e-04,2.125588e-04,0.000520,0.000516,0.000557,4.105257e-04,4.283622e-04,4.876879e-04,5.125729e-04
14241,2018-12-28 00:00:00-07:00,0.000416,0.000421,0.000420,0.000420,0.000404,0.000397,0.000237,0.000144,0.000320,...,5.797572e-05,2.086436e-05,3.461967e-05,0.000000,0.000000,0.000000,1.449955e-07,2.026077e-07,8.678600e-08,2.029225e-09
14242,2018-12-29 00:00:00-07:00,0.012657,0.014018,0.017795,0.017795,0.017351,0.016241,0.006376,0.002629,0.015110,...,1.372970e-07,1.154708e-06,2.652976e-06,0.000467,0.000451,0.000399,1.812913e-04,2.044448e-04,2.481727e-04,3.005849e-04
14243,2018-12-30 00:00:00-07:00,0.006537,0.006436,0.006874,0.006874,0.006573,0.006468,0.003893,0.004057,0.005056,...,5.515527e-04,8.730351e-04,8.369709e-04,0.004444,0.004347,0.005098,2.701479e-03,3.155705e-03,2.954687e-03,3.329640e-03


In [19]:
# Convert the first row (excluding the first column) to integers and truncate the decimals
precipitation_daily_df.iloc[0, 1:] = precipitation_daily_df.iloc[0, 1:].astype(int).astype(str).str.split('.').str[0]
temperature_daily_max_df.iloc[0, 1:] = temperature_daily_max_df.iloc[0, 1:].astype(int).astype(str).str.split('.').str[0]
temperature_daily_min_df.iloc[0, 1:] = temperature_daily_min_df.iloc[0, 1:].astype(int).astype(str).str.split('.').str[0]

In [20]:
# Move the first row into the index and the first column into the index
precipitation_daily_df = precipitation_daily_df.rename(columns=precipitation_daily_df.iloc[0]).drop(precipitation_daily_df.index[0]).reset_index(drop=True)
precipitation_daily_df = precipitation_daily_df.set_index('time')

temperature_daily_max_df = temperature_daily_max_df.rename(columns=temperature_daily_max_df.iloc[0]).drop(temperature_daily_max_df.index[0]).reset_index(drop=True)
temperature_daily_max_df = temperature_daily_max_df.set_index('time')

temperature_daily_min_df = temperature_daily_min_df.rename(columns=temperature_daily_min_df.iloc[0]).drop(temperature_daily_min_df.index[0]).reset_index(drop=True)
temperature_daily_min_df = temperature_daily_min_df.set_index('time')


In [21]:
# ensure index values are int
precipitation_daily_df.columns = precipitation_daily_df.columns.astype(int)
temperature_daily_max_df.columns = temperature_daily_max_df.columns.astype(int)
temperature_daily_min_df.columns = temperature_daily_min_df.columns.astype(int)

In [22]:
temperature_daily_min_df

ID,95965,95966,95968,95969,95970,95974,95977,95979,95980,95981,...,97351,97352,97353,96100,96102,96139,96959,96978,97007,97013
time,,,,,,,,,,,,,,,,,,,,,
1980-01-01 00:00:00-07:00,-4.077185,-4.063258,-4.249374,-4.249374,-4.204485,-4.264989,-5.604952,-5.902067,-3.985034,-6.71312,...,-14.488849,-14.164856,-14.419731,-12.799255,-12.747369,-11.618834,-13.339889,-12.941555,-12.668878,-12.203747
1980-01-02 00:00:00-07:00,-5.943811,-6.297486,-5.732376,-5.732376,-5.586266,-6.269331,-11.914492,-10.657153,-5.207447,-11.546248,...,-13.12762,-11.295785,-11.891782,-15.039352,-14.976529,-13.297913,-14.946922,-15.563313,-13.501598,-13.259534
1980-01-03 00:00:00-07:00,-9.067291,-9.300563,-8.556793,-8.556793,-8.627795,-9.166342,-10.194782,-11.097475,-8.985792,-12.034689,...,-11.323739,-10.497956,-10.703948,-16.74234,-16.541409,-15.374904,-13.703654,-13.409696,-15.595657,-15.650296
1980-01-04 00:00:00-07:00,-10.347761,-10.295909,-9.94014,-9.94014,-9.94809,-10.259353,-11.956533,-15.375196,-10.019004,-16.416164,...,-20.12076,-16.849611,-18.602684,-20.284668,-20.323328,-18.987097,-21.491861,-21.012577,-19.720421,-19.468876
1980-01-05 00:00:00-07:00,-24.12735,-24.19482,-24.529449,-24.529449,-24.622162,-24.603601,-25.230572,-24.330433,-25.08963,-24.374279,...,-22.032393,-22.393494,-22.914274,-23.847809,-23.897152,-23.517336,-24.361767,-24.418287,-24.358967,-24.309139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-27 00:00:00-07:00,-11.546169,-11.775364,-11.292328,-11.292328,-11.413207,-11.906411,-18.472185,-14.699361,-12.02269,-15.624222,...,-12.349693,-11.688888,-11.28916,-13.942719,-13.961031,-13.226499,-14.263212,-13.570544,-13.573983,-14.004589
2018-12-28 00:00:00-07:00,-14.43239,-14.495928,-14.042953,-14.042953,-14.222805,-14.596349,-22.497684,-19.001493,-15.129632,-19.75959,...,-18.179014,-15.736199,-16.178595,-20.362595,-20.499685,-19.657385,-21.949774,-20.983965,-21.06526,-20.906525
2018-12-29 00:00:00-07:00,-10.011073,-10.178868,-10.903061,-10.903061,-10.89954,-10.82236,-9.913393,-8.737276,-10.881788,-8.82144,...,-9.295745,-6.471768,-7.111849,-7.293686,-7.151507,-5.480192,-5.066916,-5.161694,-5.626347,-6.144136


In [23]:
# Read the parameters file into a DataFrame to get order of stations
data_file = '/home/paulc600/scratch/PRMS/02_directory/inputs/myparam.param'
id_order = pd.read_csv(data_file, delimiter='\t')

In [24]:
# Trim the DataFrame to the only contain station id order
start_row = 98058
end_row = start_row + len(precipitation_daily_df.columns)
id_order = id_order.iloc[start_row:end_row]

In [25]:
id_order

,Written by Bandit version 0.8.7
98058,95965
98059,95966
98060,95968
98061,95969
98062,95970
...,...
98962,97349
98963,97350
98964,97351
98965,97352


In [26]:
# Convert the values in the first column of trimmed_data_df to integers
id_order.iloc[:, 0] = id_order.iloc[:, 0].astype(int)

In [27]:
# Create a list of column names from the first column of id_order
column_names = id_order.iloc[:, 0].tolist()

In [28]:
# Reorder the columns of each DataFrame based on the list of column names
precipitation_daily_df = precipitation_daily_df[column_names]
temperature_daily_max_df = temperature_daily_max_df[column_names]
temperature_daily_min_df = temperature_daily_min_df[column_names]

In [29]:
# Convert the datetime index to a string representation with 'YYYY-MM-DD' format
precipitation_daily_df.index = precipitation_daily_df.index.strftime('%Y-%m-%d')
temperature_daily_max_df.index = temperature_daily_max_df.index.strftime('%Y-%m-%d')
temperature_daily_min_df.index = temperature_daily_min_df.index.strftime('%Y-%m-%d')

In [30]:
# Format the data to display two significant figures
precipitation_daily_df = precipitation_daily_df.applymap(lambda x: '{:.2g}'.format(x))
temperature_daily_max_df = temperature_daily_max_df.applymap(lambda x: '{:.2g}'.format(x))
temperature_daily_min_df = temperature_daily_min_df.applymap(lambda x: '{:.2g}'.format(x))

In [31]:
precipitation_daily_df

ID,95965,95966,95968,95969,95970,95974,95977,95979,95980,95981,...,97344,97345,97346,97347,97348,97349,97350,97351,97352,97353
time,,,,,,,,,,,,,,,,,,,,,
1980-01-01,0.00074,0.00077,0.00094,0.00094,0.00089,0.00085,0.00028,0.00012,0.00063,9.4e-05,...,0,0,4.5e-07,2e-06,3e-06,1.4e-06,2.5e-06,3.6e-06,6.3e-07,6.3e-07
1980-01-02,0.0006,0.00073,0.00097,0.00097,0.00098,0.00086,0.00023,0.00021,0.001,0.00015,...,1.3e-05,8.6e-06,3.1e-05,7.1e-05,7.9e-05,8e-05,8.8e-05,8.6e-05,4.4e-05,5.3e-05
1980-01-03,0.0045,0.0044,0.0052,0.0052,0.0049,0.0046,0.00046,0.00037,0.0034,0.00021,...,0.00016,5.4e-05,3.5e-05,6.6e-05,8.5e-05,8.3e-05,0.00014,0.00014,0.00027,0.00027
1980-01-04,0.00048,0.00045,0.00037,0.00037,0.00034,0.00034,0.00021,0.00016,0.00019,9.4e-05,...,0.0004,0.00022,0.00018,0.00021,0.00019,0.00018,0.00013,0.00014,0.00017,8.4e-05
1980-01-05,0.017,0.017,0.018,0.018,0.017,0.017,0.014,0.012,0.017,0.012,...,0.0046,0.0034,0.003,0.0034,0.0034,0.0032,0.0035,0.0036,0.0052,0.0042
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-27,0.00068,0.00064,0.00066,0.00066,0.00067,0.00063,0.0004,0.00042,0.00074,0.00036,...,0.00046,0.00046,0.00035,0.00019,0.00019,0.0002,0.0002,0.00019,0.00025,0.00021
2018-12-28,0.00042,0.00042,0.00042,0.00042,0.0004,0.0004,0.00024,0.00014,0.00032,0.00014,...,2.7e-05,5.6e-05,7.9e-05,8.1e-05,7.5e-05,7.3e-05,5.7e-05,5.8e-05,2.1e-05,3.5e-05
2018-12-29,0.013,0.014,0.018,0.018,0.017,0.016,0.0064,0.0026,0.015,0.0026,...,5.5e-06,5.6e-06,1.1e-06,0,0,0,1.3e-07,1.4e-07,1.2e-06,2.7e-06


In [39]:
# Write precipitation_daily_df to a text file
with open(precipitation_file_path, 'w') as file:
    # Write custom headers
    file.write('written by paul\n')
    file.write(f'prcp {len(column_names)}\n')
    file.write('################################################\n')
 
    # Write data
    for date_str, row in zip(precipitation_daily_df.index, precipitation_daily_df.itertuples(index=False)):
        date_parts = date_str.split('-')
        year, month, day = date_parts[0], date_parts[1], date_parts[2]
        
        # Create a list with year, month, day, and three zeros, followed by data
        output_row = [year, month, day, '0', '0', '0'] + [str(val) for val in row]
        
        # Write the row to the file
        file.write('\t'.join(output_row) + '\n')

In [43]:
# Write tmax to a text file
with open(temperature_max_file_path, 'w') as file:
    # Write custom headers
    file.write('written by paul\n')
    file.write(f'tmax {len(column_names)}\n')
    file.write('################################################\n')

   
    # Write data
    for date_str, row in zip(temperature_daily_max_df.index, temperature_daily_max_df.itertuples(index=False)):
        date_parts = date_str.split('-')
        year, month, day = date_parts[0], date_parts[1], date_parts[2]
        
        # Create a list with year, month, day, and three zeros, followed by data
        output_row = [year, month, day, '0', '0', '0'] + [str(val) for val in row]
        
        # Write the row to the file
        file.write('\t'.join(output_row) + '\n')

In [44]:
# Write tmin to a text file
with open(temperature_min_file_path, 'w') as file:
    # Write custom headers
    file.write('written by paul\n')
    file.write(f'tmin {len(column_names)}\n')
    file.write('################################################\n')

   
    # Write data
    for date_str, row in zip(temperature_daily_min_df.index, temperature_daily_min_df.itertuples(index=False)):
        date_parts = date_str.split('-')
        year, month, day = date_parts[0], date_parts[1], date_parts[2]
        
        # Create a list with year, month, day, and three zeros, followed by data
        output_row = [year, month, day, '0', '0', '0'] + [str(val) for val in row]
        
        # Write the row to the file
        file.write('\t'.join(output_row) + '\n')